In [1]:
import  numpy as np
import pandas as pd
import ast
import nltk
import pickle as pk

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
credits.rename(columns = {'movie_id' : 'id'}, inplace=True)

In [4]:
movies = movies.merge(credits, on='id')

In [5]:
movies = movies[['id','original_title','genres','keywords','overview','production_companies','cast','crew','release_date','runtime','homepage']]

In [6]:
def conv(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [7]:
movies['genres'] = movies['genres'].apply(conv)

In [8]:
movies['keywords'] = movies['keywords'].apply(conv)

In [9]:
movies['production_companies'] = movies['production_companies'].apply(conv)

In [10]:
def convert(obj):
    L=[]
    c=0
    for i in ast.literal_eval(obj):
        if c!=5:
            L.append(i['name'])
            c+=1
        else:
            break
    return L

In [11]:
movies['cast'] = movies['cast'].apply(convert)

In [12]:
movies['producer'] = movies['crew']

In [13]:
movies.rename(columns = {'crew' : 'director'}, inplace=True)

In [14]:
def dp(obj,job):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']==job:
            L.append(i['name'])
            break
    return L

In [15]:
movies['director'] = movies['director'].apply(lambda x:dp(x,'Director'))

In [16]:
movies['producer'] = movies['producer'].apply(lambda x:dp(x,'Producer'))

In [17]:
movies["tag"] = movies['overview'].apply(lambda x:str(x).split()) + movies['genres'].apply(lambda x:[i.replace(" ","") for i in x]) + movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x]) + movies['production_companies'].apply(lambda x:[i.replace(" ","") for i in x]) + movies['cast'].apply(lambda x:[i.replace(" ","") for i in x]) + movies['director'].apply(lambda x:[i.replace(" ","") for i in x]) + movies['producer'].apply(lambda x:[i.replace(" ","") for i in x])

In [18]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [19]:
movies['tag'] = movies['tag'].apply(lambda x:[ps.stem(i) for i in x])

In [20]:
movies['tag'] = movies['tag'].apply(lambda x:' '.join(x))

In [21]:
movies['genres'] = movies['genres'].apply(lambda x:', '.join(x))

In [22]:
movies['production_companies'] = movies['production_companies'].apply(lambda x:', '.join(x))

In [23]:
movies['cast'] = movies['cast'].apply(lambda x:', '.join(x))

In [24]:
movies['director'] = movies['director'].apply(lambda x:''.join(x))

In [25]:
movies['producer'] = movies['producer'].apply(lambda x:''.join(x))

In [26]:
del movies['keywords']

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2*len(movies), stop_words='english')

In [28]:
vec = cv.fit_transform(movies['tag']).toarray()

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
sim = cosine_similarity(vec)

In [31]:
def recommend(mov,count):
    a = 6 + count*5
    mov_index = movies[movies['original_title'] == mov].index[0]
    mov_list = sorted(list(enumerate(sim[mov_index])),reverse=True,key=lambda x:x[1])[1:a]
    
    for i in mov_list:
        print(movies['original_title'][i[0]])

In [32]:
recommend("Superman Returns",0)

Superman II
Superman IV: The Quest for Peace
Superman
Superman III
Man of Steel


In [33]:
pk.dump(movies.to_dict(),open('movies_dictionary.pkl','wb'))

In [34]:
pk.dump(sim,open('sim.pkl','wb'))

In [35]:
movies

,id,original_title,genres,overview,production_companies,cast,director,release_date,runtime,homepage,producer,tag
0,19995,Avatar,"Action, Adventure, Fantasy, Science Fiction","In the 22nd century, a paraplegic Marine is di...","Ingenious Film Partners, Twentieth Century Fox...","Sam Worthington, Zoe Saldana, Sigourney Weaver...",James Cameron,2009-12-10,162.0,http://www.avatarmovie.com/,James Cameron,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"Adventure, Fantasy, Action","Captain Barbossa, long believed to be dead, ha...","Walt Disney Pictures, Jerry Bruckheimer Films,...","Johnny Depp, Orlando Bloom, Keira Knightley, S...",Gore Verbinski,2007-05-19,169.0,http://disney.go.com/disneypictures/pirates/,Jerry Bruckheimer,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,"Action, Adventure, Crime",A cryptic message from Bond’s past sends him o...,"Columbia Pictures, Danjaq, B24","Daniel Craig, Christoph Waltz, Léa Seydoux, Ra...",Sam Mendes,2015-10-26,148.0,http://www.sonypictures.com/movies/spectre/,Barbara Broccoli,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,"Action, Crime, Drama, Thriller",Following the death of District Attorney Harve...,"Legendary Pictures, Warner Bros., DC Entertain...","Christian Bale, Michael Caine, Gary Oldman, An...",Christopher Nolan,2012-07-16,165.0,http://www.thedarkknightrises.com/,Charles Roven,follow the death of district attorney harvey d...
4,49529,John Carter,"Action, Adventure, Science Fiction","John Carter is a war-weary, former military ca...",Walt Disney Pictures,"Taylor Kitsch, Lynn Collins, Samantha Morton, ...",Andrew Stanton,2012-03-07,132.0,http://movies.disney.com/john-carter,Colin Wilson,"john carter is a war-weary, former militari ca..."
...,...,...,...,...,...,...,...,...,...,...,...,...
4798,9367,El Mariachi,"Action, Crime, Thriller",El Mariachi just wants to play his guitar and ...,Columbia Pictures,"Carlos Gallardo, Jaime de Hoyos, Peter Marquar...",Robert Rodriguez,1992-09-04,81.0,NaN,Robert Rodriguez,el mariachi just want to play hi guitar and ca...
4799,72766,Newlyweds,"Comedy, Romance",A newlywed couple's honeymoon is upended by th...,,"Edward Burns, Kerry Bishé, Marsha Dietlein, Ca...",Edward Burns,2011-12-26,85.0,NaN,Edward Burns,a newlyw couple' honeymoon is upend by the arr...
4800,231617,"Signed, Sealed, Delivered","Comedy, Drama, Romance, TV Movie","""Signed, Sealed, Delivered"" introduces a dedic...","Front Street Pictures, Muse Entertainment Ente...","Eric Mabius, Kristin Booth, Crystal Lowe, Geof...",Scott Smith,2013-10-13,120.0,http://www.hallmarkchannel.com/signedsealeddel...,Harvey Kahn,"""signed, sealed, delivered"" introduc a dedic q..."
4801,126186,Shanghai Calling,,When ambitious New York attorney Sam is sent t...,,"Daniel Henney, Eliza Coupe, Bill Paxton, Alan ...",Daniel Hsia,2012-05-03,98.0,http://shanghaicalling.com/,,when ambiti new york attorney sam is sent to s...
